## Увод у регресију - Лекција 1

#### Стављање у перспективу

✅ Постоји много врста метода регресије, а коју ћете изабрати зависи од одговора који тражите. Ако желите да предвидите вероватну висину особе одређеног узраста, користили бисте `линеарну регресију`, јер тражите **нумеричку вредност**. Ако вас занима да ли одређена врста кухиње треба да се сматра веганском или не, тражите **категоријску класификацију**, па бисте користили `логистичку регресију`. О логистичкој регресији ћете више научити касније. Размислите мало о питањима која можете поставити подацима и који од ових метода би био прикладнији.

У овом делу, радићете са [малим сетом података о дијабетесу](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Замислите да желите да тестирате третман за пацијенте са дијабетесом. Модели машинског учења могу вам помоћи да одредите који пацијенти би боље реаговали на третман, на основу комбинација променљивих. Чак и веома основни модел регресије, када се визуализује, може показати информације о променљивим које би вам помогле да организујете теоријске клиничке студије.

С тим речима, хајде да започнемо овај задатак!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Илустрација: @allison_horst</figcaption>

<!--![Илустрација: \@allison_horst](../../../../../../2-Regression/1-Tools/images/encouRage.jpg)<br>Илустрација: @allison_horst-->


## 1. Учитавање нашег алата

За овај задатак биће нам потребни следећи пакети:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) је [збирка R пакета](https://www.tidyverse.org/packages) осмишљена да учини науку о подацима бржом, лакшом и забавнијом!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) је [оквир пакета](https://www.tidymodels.org/packages/) за моделирање и машинско учење.

Можете их инсталирати на следећи начин:

`install.packages(c("tidyverse", "tidymodels"))`

Скрипта испод проверава да ли имате инсталиране пакете потребне за завршетак овог модула и инсталира их уколико неки недостају.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Сада, хајде да учитамо ове сјајне пакете и учинимо их доступним у нашој тренутној R сесији. (Ово је само за илустрацију, `pacman::p_load()` је то већ урадио за вас)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Дијабетес скуп података

У овом задатку, применићемо наше вештине регресије правећи предвиђања на скупу података о дијабетесу. [Скуп података о дијабетесу](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) садржи `442 узорка` података о дијабетесу, са 10 предикторских променљивих: `године`, `пол`, `индекс телесне масе`, `просечан крвни притисак` и `шест мерења крвног серума`, као и излазну променљиву `y`: квантитативну меру напретка болести годину дана након почетног стања.

|Број опсервација|442|
|----------------|:---|
|Број предиктора|Првих 10 колона су нумерички предиктори|
|Излаз/Циљ|Колона 11 је квантитативна мера напретка болести годину дана након почетног стања|
|Информације о предикторима|- године живота
||- пол
||- bmi индекс телесне масе
||- bp просечан крвни притисак
||- s1 tc, укупни серумски холестерол
||- s2 ldl, липопротеини ниске густине
||- s3 hdl, липопротеини високе густине
||- s4 tch, укупни холестерол / HDL
||- s5 ltg, могуће логаритам нивоа триглицерида у серуму
||- s6 glu, ниво шећера у крви|

> 🎓 Запамтите, ово је надгледано учење, и потребан нам је именовани циљ 'y'.

Пре него што можете манипулисати подацима у R-у, потребно је да увезете податке у меморију R-а или успоставите везу са подацима коју R може користити за приступ подацима на даљину.

> Пакет [readr](https://readr.tidyverse.org/), који је део Tidyverse-а, пружа брз и једноставан начин за читање правоугаоних података у R.

Сада, учитајмо скуп података о дијабетесу који је доступан на овом URL-у: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Такође, извршићемо проверу исправности наших података користећи `glimpse()` и приказати првих 5 редова користећи `slice()`.

Пре него што наставимо даље, представићемо нешто што ћете често сусретати у R коду 🥁🥁: оператор цеви `%>%`

Оператор цеви (`%>%`) извршава операције у логичком низу тако што прослеђује објекат функцији или изразу. Можете замислити оператор цеви као да у коду каже "и онда".


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` нам показује да овај податак има 442 реда и 11 колона, при чему су све колоне типа податка `double`.

<br>

> glimpse() и slice() су функције у [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, део Tidyverse-а, је граматика за манипулацију подацима која пружа конзистентан сет глагола који вам помажу да решите најчешће изазове у манипулацији подацима.

<br>

Сада када имамо податке, усмерићемо се на једну карактеристику (`bmi`) коју ћемо циљати за ову вежбу. Ово ће захтевати да изаберемо жељене колоне. Па, како то можемо урадити?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) нам омогућава да *изаберемо* (и опционално преименујемо) колоне у оквиру података.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Тренинг и тестирање података

У надгледаном учењу је уобичајена пракса да се *подели* подаци на два подскупа; (обично већи) скуп за тренирање модела и мањи "резервни" скуп за проверу како је модел функционисао.

Сада када имамо припремљене податке, можемо видети да ли машина може помоћи у одређивању логичне поделе између бројева у овом скупу података. Можемо користити пакет [rsample](https://tidymodels.github.io/rsample/), који је део оквира Tidymodels, да креирамо објекат који садржи информације о *начину* поделе података, а затим још две rsample функције за издвајање креираних скупова за тренирање и тестирање:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Обучите модел линеарне регресије помоћу Tidymodels

Сада смо спремни да обучимо наш модел!

У Tidymodels-у, модели се специфицирају помоћу `parsnip()` кроз дефинисање три концепта:

-   **Тип модела** разликује моделе као што су линеарна регресија, логистичка регресија, модели одлуке стабла и тако даље.

-   **Режим модела** укључује уобичајене опције као што су регресија и класификација; неки типови модела подржавају обе опције, док неки имају само један режим.

-   **Енџин модела** је рачунарски алат који ће се користити за прилагођавање модела. Често су то R пакети, као што су **`"lm"`** или **`"ranger"`**

Ове информације о моделу се чувају у спецификацији модела, па хајде да направимо једну!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Након што је модел *одређен*, модел може бити `процењен` или `трениран` коришћењем функције [`fit()`](https://parsnip.tidymodels.org/reference/fit.html), обично уз помоћ формуле и неких података.

`y ~ .` значи да ћемо прилагодити `y` као предвиђену вредност/циљ, објашњену свим предикторима/карактеристикама, односно `.` (у овом случају, имамо само један предиктор: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Из модела можемо видети коефицијенте који су научени током тренинга. Они представљају коефицијенте линије најбољег прилагођавања која нам даје најмању укупну грешку између стварне и предвиђене променљиве.
<br>

## 5. Направите предвиђања на тест сету

Сада када смо обучили модел, можемо га користити за предвиђање прогресије болести y за тестни скуп података користећи [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Ово ће се користити за цртање линије између група података.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Ура! 💃🕺 Управо смо обучили модел и користили га за прављење предвиђања!

Када правимо предвиђања, конвенција tidymodels-а је да увек производимо tibble/data frame резултата са стандардизованим именима колона. Ово олакшава комбиновање оригиналних података и предвиђања у употребљивом формату за наредне операције као што је креирање графикона.

`dplyr::bind_cols()` ефикасно спаја више data frame-ова по колонама.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Приказивање резултата модела

Сада је време да ово видимо визуелно 📈. Направићемо расејани графикон свих `y` и `bmi` вредности из тест скупа, а затим ћемо користити предвиђања да нацртамо линију на најприкладнијем месту, између група података модела.

R има неколико система за прављење графикона, али `ggplot2` је један од најелегантнијих и најсвестранијих. Овај пакет вам омогућава да креирате графиконе **комбиновањем независних компоненти**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Размислите мало о томе шта се овде дешава. Права линија пролази кроз много малих тачака података, али шта она заправо ради? Можете ли да видите како би требало да будете у могућности да користите ову линију за предвиђање где би нова, непозната тачка података требало да се уклопи у односу на y осу графикона? Покушајте да речима опишете практичну употребу овог модела.

Честитамо, направили сте свој први модел линеарне регресије, креирали предвиђање са њим и приказали га на графикону!



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
